In [1]:
import numpy as np
import keras
from keras import layers

## Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Conmmetary
1. 이미지를 [0, 1] 범위로 스케일링하는 이유 :
    - 본 이미지 픽셀 값은 0부터 255 사이의 정수로 구성되어 있는데, 이를 255로 나누어 0과 1 사이의 실수로 변환하면 모든 픽셀 값의 범위를 동일하게 조정
    - 이러한 정규화 과정을 통해 모델의 수렴 속도를 높이고, 더 나은 학습 결과를 얻을 수 있다.

2. 이미지의 형태를 (28, 28, 1)로 만드는 이유 :
    - 이미지의 높이가 28, 너비가 28, 채널 수가 1
    - 대부분의 딥러닝 프레임워크는 입력 데이터의 차원을 명확히 요구하는데, 특히 컨볼루션 신경망(CNN)과 같은 이미지 처리 모델은 이미지의 채널 정보를 필요로 함.
    - 흑백 이미지라 하더라도 명시적으로 채널 차원을 추가함으로써 (28, 28) 형태의 이미지를 (28, 28, 1) 형태로 확장하여 모델이 이를 정확하게 인식하고 처리할 수 있도록 함.

## Build the model

In [14]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(3, 3)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 6, 6, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         5,770 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,586 (96.04 KB)

 Trainable params: 24,586 (96.04 KB)

 Non-trainable params: 0 (0.00 B)

## Commentary

1. Conv2D
- filters – int, the dimension of the output space
    - the number of filters in the convolution.
    - the depth of the hidden layer
- padding – string, either "valid" or "same" (case-insensitive). 
    - "valid" means no padding. 
    - "same" results in padding evenly to the left/right or up/down of the input. 
    - When padding="same" and strides=1, the output has the same size as the input.

## Train the model

In [12]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5052 - loss: 1.5052 - val_accuracy: 0.9352 - val_loss: 0.2675
Epoch 2/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8518 - loss: 0.4849 - val_accuracy: 0.9565 - val_loss: 0.1655
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8915 - loss: 0.3548 - val_accuracy: 0.9660 - val_loss: 0.1235
Epoch 4/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9056 - loss: 0.3079 - val_accuracy: 0.9690 - val_loss: 0.1114
Epoch 5/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9173 - loss: 0.2724 - val_accuracy: 0.9720 - val_loss: 0.0965
Epoch 6/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9253 - loss: 0.2468 - val_accuracy: 0.9743 - val_loss: 0.0869
Epoch 7/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9271 - loss: 0.2392 - val_accuracy: 0.9762 - val_loss: 0.0828
Epoch 8/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9344 - loss: 0.2168 - val_accuracy: 0.

## Evaluate the trained model

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.07990085333585739
Test accuracy: 0.9769999980926514
